In [1]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import time

In [2]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')

base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

base = base[base.price > 10]
base = base.loc[base.price < 350000]

valores = {'vehicleType': 'limousine', 'gearbox': 'manuell',
           'model': 'golf', 'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
 
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])],remainder='passthrough')
previsores = onehotencorder.fit_transform(previsores).toarray()

In [ ]:
def criar_rede(unit1, unit2, drop1, drop2, kernel_initializer1, kernel_initializer2, optimizer):
    
    regressor = Sequential()
    
    regressor.add(Dense(units = unit1, activation = 'relu',  kernel_initializer = kernel_initializer1, input_dim = 316))
    
    classificador.add(Dropout(drop1))
    
    regressor.add(Dense(units = unit2, activation = 'relu',  kernel_initializer = kernel_initializer2))
    
    classificador.add(Dropout(drop2))
    
    regressor.add(Dense(units = 1, activation = 'linear'))
    
    regressor.compile(loss = 'mean_absolute_error', optimizer = optimizer,
                      metrics = ['mean_absolute_error'])
    
    return regressor

In [ ]:
classificador = KerasRegressor(build_fn = criarRede)
parametros = {'batch_size': [300],
              'epochs': [100],
              #'callbacks': [ClearMemory()],
              'drop1': [0.2, 0.3],
              'drop2': [0.2, 0.3],
              'unit1': [158],
              'unit2': [158],
              'kernel_initializer1': ['random_uniform'],
              'kernel_initializer2': ['random_uniform'],
              'optimizer': ['RMSprop', 'adam', 'sgd']
              #'clipvalue': [0.5, 0.1, 0.05]
             }

In [ ]:
grid_search = GridSearchCV(estimator = classificador,
                           param_grid = parametros,
                           scoring = 'neg_mean_absolute_error',
                           cv = 4,
                           n_jobs=1)

In [ ]:
grid_search = grid_search.fit(previsores, preco_real)